In [22]:
pip install pydeequ==1.0.1 

Note: you may need to restart the kernel to use updated packages.


In [23]:
pip install py4j

Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install pyspark==3.0.3     

Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install qvd

Note: you may need to restart the kernel to use updated packages.


In [26]:
from pyspark.sql import SparkSession, Row
import pydeequ
import pandas as pd
import re
from qvd import qvd_reader
from pyspark.sql.types import IntegerType
from pydeequ.checks import *
from pydeequ.verification import *
import numpy as np

In [27]:
spark = (SparkSession
    .builder
    .appName("Example") \
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .config("spark.driver.memory", "16g")\
    .config("spark.executor.memory", "4g") \
    .getOrCreate())
check = Check(spark, CheckLevel.Warning, "Review Check")

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:736)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:271)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1120)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1106)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:547)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1574)
	at org.apache.spark.SparkContext.$anonfun$new$12(SparkContext.scala:498)
	at org.apache.spark.SparkContext.$anonfun$new$12$adapted(SparkContext.scala:498)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:498)
	at org.apache.spark.api.java.JavaSparkC

In [ ]:
df_param = pd.read_excel("Parametros.xlsx",sheet_name="Validaciones")
#df_Person_Person = pd.read_csv('dbo_Person_Person.csv')
#df_address = pd.read_csv('dbo_Address.csv')

In [ ]:
df_param_ = df_param.loc[df_param['Tipo Validacion'] == 'Formato']
tablas = df_param_.drop_duplicates(subset=['Esquema', 'Tabla'])
name_tablas = tablas.reset_index()
name_tabla = name_tablas.iloc[0]["Tabla"]
tablas = np.array(tablas["Esquema"]+"_"+tablas["Tabla"])
tablas_campo = df_param_["Esquema"]+"_"+df_param_["Tabla"]+"."+df_param_["Campo"]
df_param_["Valores"]=df_param_["Valores"].replace("TipoDato", "hasDataType")
df_param_["Valores"]=df_param_["Valores"].replace("MinLength", "hasMinLength")
df_param_["Valores"]=df_param_["Valores"].replace("MaxLength", "hasMaxLength")
df_param_["Valores"]=df_param_["Valores"].replace("Email", "containsEmail")
df_param_["Valores"]=df_param_["Valores"].replace("MaxNumRegistros", "hasSize")
df_param_["Valores"]=df_param_["Valores"].replace("CreditCardNumber", "containsCreditCardNumber")
df_param_["Valores"]=df_param_["Valores"].replace("URL", "containsURL")
df_param_["Valores"]=df_param_["Valores"].replace("SocialSecurityNumber", "containsSocialSecurityNumber")

In [ ]:
df_param_

In [ ]:
df_param_ = df_param_.loc[df_param_['Tabla'] == name_tabla]
df_param_

In [ ]:
#data = {}
data = pd.read_csv(tablas[0]+'.csv')

In [ ]:
result_df = {}
result_df_ = pd.DataFrame()
for i in range(len(df_param_)):

  if df_param_.iloc[i]["Valores"] == 'hasMaxLength':
    check_ = Check(spark, CheckLevel.Warning, "hasMaxLength").hasMaxLength(df_param_.iloc[i]["Campo"],lambda x: x == df_param_.iloc[i]["Formato"])
    result_ = VerificationSuite(spark).onData(data).addCheck(check_).run()
    result_df[i] = VerificationResult.checkResultsAsDataFrame(spark, result_, pandas=True)

  elif df_param_.iloc[i]["Valores"] == 'hasMinLength':
    check_ = Check(spark, CheckLevel.Warning, "hasMinLength").hasMinLength(df_param_.iloc[i]["Campo"],lambda x: x == df_param_.iloc[i]["Formato"])
    result_ = VerificationSuite(spark).onData(data).addCheck(check_).run()
    result_df[i] = VerificationResult.checkResultsAsDataFrame(spark, result_, pandas=True)

  elif df_param_.iloc[i]["Valores"] == 'containsEmail':
    check_ = Check(spark, CheckLevel.Warning, "containsEmail").containsEmail(df_param_.iloc[i]["Campo"])
    result_ = VerificationSuite(spark).onData(data).addCheck(check_).run()
    result_df[i] = VerificationResult.checkResultsAsDataFrame(spark, result_, pandas=True)

  elif df_param_.iloc[i]["Valores"] == 'containsURL':
    check_ = Check(spark, CheckLevel.Warning, "containsURL").containsURL(df_param_.iloc[i]["Campo"])
    result_ = VerificationSuite(spark).onData(data).addCheck(check_).run()
    result_df[i] = VerificationResult.checkResultsAsDataFrame(spark, result_, pandas=True)

  #elif df_param_.iloc[i]["Valores"] == 'hasSize':
    #check_ = Check(spark, CheckLevel.Warning, "hasSize").hasSize(df_param_.iloc[i]["Campo"], lambda x: x <= df_param_.iloc[i]["Formato"])
    #result_ = VerificationSuite(spark).onData(data).addCheck(check_).run()
    #result_df[i] = VerificationResult.checkResultsAsDataFrame(spark, result_, pandas=True)

  #elif df_param_.iloc[i]["Valores"] == 'hasDataType':
    #check_ = Check(spark, CheckLevel.Warning, "hasDataType").hasDataType(df_param_.iloc[i]["Campo"], df_param_.iloc[i]["Formato"])
    #result_ = VerificationSuite(spark).onData(data).addCheck(check_).run()
    #result_df[i] = VerificationResult.checkResultsAsDataFrame(spark, result_)

  elif df_param_.iloc[i]["Valores"] == 'containsCreditCardNumber':
    check_ = Check(spark, CheckLevel.Warning, "containsCreditCardNumber").containsCreditCardNumber(df_param_.iloc[i]["Campo"])
    result_ = VerificationSuite(spark).onData(data).addCheck(check_).run()
    result_df[i] = VerificationResult.checkResultsAsDataFrame(spark, result_, pandas=True)

  elif df_param_.iloc[i]["Valores"] == 'containsSocialSecurityNumber':
    check_ = Check(spark, CheckLevel.Warning, "containsSocialSecurityNumber").containsSocialSecurityNumber(df_param_.iloc[i]["Campo"])
    result_ = VerificationSuite(spark).onData(data).addCheck(check_).run()
    result_df[i] = VerificationResult.checkResultsAsDataFrame(spark, result_, pandas=True)

In [ ]:
result_df_= pd.concat(result_df)
result_df_

In [ ]:

print("Longest length is:\n",df_Person_Person.Password.str.len().max())

In [ ]:

print("shortes length is:\n",df_Person_Person.Password.str.len().min())

In [ ]:
data.Email[0] = '333@333.com'
data.Email[1] = '333@333.333 '
data.Email[2] = 'jonatan@gmail.com'

In [ ]:
data.Email